<a href="https://colab.research.google.com/github/cyberust/HumanSynergyAnalysis/blob/main/HumanSynergySemanticAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
# ======================================================================
#
#  人的シナジー分析システム V3 - 最終安定版
#
#  改善点:
#  - 全てのパラメータを冒頭のCONFIGで一元管理
#  - 全ての処理を単一のパイプラインで実行し、一貫性を担保
#  - 可視化とファイル保存を分離し、Colabのリソース問題を根本的に解決
#
# ======================================================================

# ===============================================================
# STEP 1: ライブラリのインストール
# ===============================================================
!pip install sentence-transformers pandas numpy scipy networkx plotly scikit-learn --quiet
print("✅ ライブラリのインストールが完了しました。")

# ===============================================================
# STEP 2: ライブラリのインポート
# ===============================================================
import os
import re
import pandas as pd
import numpy as np
import networkx as nx
import plotly.express as px
from datetime import datetime
from scipy.integrate import odeint
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')
print("✅ ライブラリのインポートが完了しました。")


# ===============================================================
# STEP 3: 分析パラメータの一元管理
# ===============================================================
CONFIG = {
    'data': {
        'base_path': "/content/drive/MyDrive/",
        'names': ["arale_cohen", "yanay_geva", "yasuyuki_sakane"],
        'display_names': ["Arale Cohen", "Yanay Geva", "Yasuyuki Sakane"]
    },
    'profile_analysis': {
        'embedding_model': 'all-MiniLM-L6-v2',
        'experience_default': 10,
        'network_keywords': ['network', 'ecosystem', 'connections', 'board member', 'partner'],
        'network_multiplier': 20
    },
    'synergy_calculation': {
        'weights': {
            'complementarity': 0.5, # スキル補完性の重要度をさらに上げる
            'experience': 0.2,
            'network': 0.2
        },
        'diversity_bonus': 1.2, # 異文化ボーナス
        'cultural_backgrounds': {'Arale Cohen': 'Western', 'Yanay Geva': 'Western', 'Yasuyuki Sakane': 'Eastern'}
    },
    'dynamic_model': {
        'alpha': np.array([0.4, 0.35, 0.45]), # 個人の基礎成長率
        'beta_multiplier': 0.1, # シナジー効果の強さ
        'K': np.array([12.0, 12.0, 12.0]), # パフォーマンス上限
        'initial_state': [1.0, 1.0, 1.0],
        'time_horizon': 24
    },
    'game_theory': {
        'synergy_value_multiplier': 5
    },
    'business_forecast': {
        'revenue_base_multiplier': 10,
        'revenue_synergy_multiplier': 5,
        'innovation_synergy_multiplier': 10,
        'market_share_cap': 40,
        'satisfaction_base': 75,
        'satisfaction_synergy_cap': 98
    },
    'output': {
        'charts_path': "/content/drive/MyDrive/synergy_analysis_charts_V3/",
        'report_path': "/content/drive/MyDrive/"
    }
}
print("✅ 分析パラメータの定義が完了しました。")


# ===============================================================
# STEP 4: 各種分析クラスの定義
# ===============================================================

class DataLoader:
    def __init__(self, config):
        self.config = config['data']

    def load_profiles(self):
        drive.mount('/content/drive', force_remount=True)
        profiles = {}
        for name, display_name in zip(self.config['names'], self.config['display_names']):
            try:
                with open(f"{self.config['base_path']}{name}_profile.txt", 'r', encoding='utf-8') as f:
                    profiles[display_name] = f.read()
            except FileNotFoundError:
                profiles[display_name] = "Profile data not available."
        return profiles

class SemanticProfileAnalyzer:
    def __init__(self, config):
        self.config = config['profile_analysis']
        self.model = SentenceTransformer(self.config['embedding_model'])

    def analyze(self, profiles_dict):
        analysis_results = {}
        names = list(profiles_dict.keys())
        texts = list(profiles_dict.values())
        embeddings = self.model.encode(texts, show_progress_bar=False)

        for i, name in enumerate(names):
            analysis_results[name] = {
                'embedding': embeddings[i],
                'experience_years': self._extract_experience(texts[i]),
                'network_strength': self._extract_network_strength(texts[i])
            }
        return analysis_results

    def _extract_experience(self, text):
        matches = re.findall(r'(\d+)\+?\s*years?', text.lower())
        return max([int(m) for m in matches]) if matches else self.config['experience_default']

    def _extract_network_strength(self, text):
        score = sum(text.lower().count(k) for k in self.config['network_keywords'])
        return min(score * self.config['network_multiplier'], 100)

class SynergyCalculator:
    def __init__(self, config):
        self.config = config['synergy_calculation']

    def calculate(self, analysis_data):
        names = list(analysis_data.keys())
        # 1. スキル補完性
        embeddings = np.array([analysis_data[n]['embedding'] for n in names])
        comp_matrix = 1 - cosine_similarity(embeddings)

        # 2. 経験シナジー
        experiences = [analysis_data[n]['experience_years'] for n in names]
        exp_matrix = np.zeros((len(names), len(names)))
        for i, j in np.ndindex(exp_matrix.shape):
            if i != j: exp_matrix[i, j] = np.exp(-0.1 * abs(experiences[i] - experiences[j]))

        # 3. ネットワーク効果
        networks = [analysis_data[n]['network_strength'] for n in names]
        net_matrix = np.zeros((len(names), len(names)))
        for i, j in np.ndindex(net_matrix.shape):
            if i != j: net_matrix[i, j] = (networks[i] * networks[j]) / 10000.0

        # 4. 総合シナジー計算
        w = self.config['weights']
        total_synergy = (w['complementarity'] * comp_matrix +
                         w['experience'] * exp_matrix +
                         w['network'] * net_matrix)

        # 5. 多様性ボーナス
        backgrounds = self.config['cultural_backgrounds']
        for i, n1 in enumerate(names):
            for j, n2 in enumerate(names):
                if backgrounds.get(n1) != backgrounds.get(n2):
                    total_synergy[i, j] *= self.config['diversity_bonus']

        np.fill_diagonal(total_synergy, 0)
        return {'total_synergy': total_synergy, 'skill_complementarity': comp_matrix}

class DynamicSystemModel:
    def __init__(self, config):
        self.config = config['dynamic_model']

    def simulate(self, synergy_matrix):
        c = self.config
        alpha = c['alpha']
        beta = synergy_matrix * c['beta_multiplier']
        K = c['K']
        t = np.linspace(0, c['time_horizon'], c['time_horizon'] * 4)
        solution = odeint(self._system_dynamics, c['initial_state'], t, args=(alpha, beta, K))
        return t, solution

    def _system_dynamics(self, state, t, alpha, beta, K):
        x = np.array(state)
        dxdt = (alpha * x + np.dot(beta, x)) * (1 - x / K)
        return dxdt

class GameTheoryAnalyzer:
    def __init__(self, config):
        self.config = config['game_theory']

    def calculate_shapley(self, analysis_data, synergy_matrix):
        from math import factorial
        from itertools import combinations

        names = list(analysis_data.keys())
        n_players = len(names)
        shapley_values = np.zeros(n_players)

        for i in range(n_players):
            for coalition in combinations(range(n_players), i):
                s_without_i = list(coalition)
                s_with_i = s_without_i + [i]
                marginal_contrib = self._get_coalition_value(s_with_i, analysis_data, synergy_matrix) - \
                                   self._get_coalition_value(s_without_i, analysis_data, synergy_matrix)
                weight = (factorial(i) * factorial(n_players - i - 1)) / factorial(n_players)
                shapley_values[i] += weight * marginal_contrib
        return shapley_values

    def _get_coalition_value(self, indices, analysis_data, synergy_matrix):
        if not indices: return 0
        names = list(analysis_data.keys())
        base_val = sum(analysis_data[names[i]]['experience_years'] +
                       analysis_data[names[i]]['network_strength'] / 10.0 for i in indices)

        synergy_val = 0
        if len(indices) > 1:
            for i in indices:
                for j in indices:
                    synergy_val += synergy_matrix[i, j]
        return base_val + synergy_val * self.config['synergy_value_multiplier']

class VisualizationGenerator:
    def __init__(self, config):
        self.config = config['data']

    def generate_all_figures(self, results):
        figures = {}
        names = self.config['display_names']

        # シナジーヒートマップ
        figures['synergy_heatmap'] = px.imshow(results['synergy']['total_synergy'], x=names, y=names, text_auto='.3f',
                                               labels=dict(color="シナジー強度"), color_continuous_scale='Blues',
                                               title='V3: 総合シナジーマトリックス')
        # 成長軌道
        df_growth = pd.DataFrame(results['simulation_solution'], columns=names)
        df_growth['Time'] = results['simulation_time_axis']
        figures['growth_trajectory'] = px.line(df_growth, x='Time', y=names, title='V3: 動的成長軌道シミュレーション')

        # ビジネス指標
        df_biz = results['business_forecast']
        fig_biz = px.line(df_biz, x='time_months', y=['revenue_growth_rate', 'innovation_index'],
                          title='V3: ビジネス指標予測（売上・イノベーション）', facet_row="variable", labels={"variable":""})
        fig_biz.update_yaxes(matches=None)
        figures['business_dashboard'] = fig_biz

        # シャプレー値
        figures['shapley_values'] = px.bar(x=names, y=results['shapley_values'], text_auto='.2f',
                                           labels={'x':'メンバー', 'y':'価値貢献度'}, title='V3: シャプレー値による価値貢献度')

        # ネットワークグラフ
        G = nx.from_numpy_array(results['synergy']['total_synergy'])
        pos = nx.spring_layout(G, k=1.5, iterations=50)
        edge_x, edge_y, edge_weights = [], [], []
        for edge in G.edges(data=True):
            x0, y0 = pos[edge[0]]
            x1, y1 = pos[edge[1]]
            edge_x.extend([x0, x1, None])
            edge_y.extend([y0, y1, None])
            edge_weights.append(edge[2]['weight'])
        edge_trace = go.Scatter(x=edge_x, y=edge_y, line=dict(width=2, color='#888'), hoverinfo='none', mode='lines')
        node_trace = go.Scatter(x=[pos[i][0] for i in G.nodes()], y=[pos[i][1] for i in G.nodes()], mode='markers+text',
                                text=[names[i] for i in G.nodes()], textposition="top center",
                                marker=dict(size=[results['analysis'][n]['network_strength']/5 for n in names],
                                            color=['#FF6B6B', '#4ECDC4', '#45B7D1']))
        fig_net = go.Figure(data=[edge_trace, node_trace], layout=go.Layout(title='V3: チーム相互作用ネットワーク', showlegend=False))
        figures['network_graph'] = fig_net

        return figures

# ===============================================================
# STEP 5: 実行パイプラインの定義と実行
# ===============================================================

class AnalysisPipeline:
    def __init__(self, config):
        self.config = config
        self.data_loader = DataLoader(config)
        self.profile_analyzer = SemanticProfileAnalyzer(config)
        self.synergy_calculator = SynergyCalculator(config)
        self.dynamic_model = DynamicSystemModel(config)
        self.game_theory_analyzer = GameTheoryAnalyzer(config)
        self.viz_generator = VisualizationGenerator(config)

    def run(self):
        print("--- パイプライン実行開始 ---")
        # 1. データ読み込み
        profiles = self.data_loader.load_profiles()
        print(f"✅ {len(profiles)}件のプロフィールを読み込みました。")

        # 2. プロフィール分析
        analysis_data = self.profile_analyzer.analyze(profiles)
        print("✅ プロフィール分析（意味ベクトル化）が完了しました。")

        # 3. シナジー計算
        synergy_results = self.synergy_calculator.calculate(analysis_data)
        print("✅ シナジーマトリクスの計算が完了しました。")

        # 4. 動的シミュレーション
        time_axis, solution = self.dynamic_model.simulate(synergy_results['total_synergy'])
        print("✅ 動的成長シミュレーションが完了しました。")

        # 5. ゲーム理論分析
        shapley_values = self.game_theory_analyzer.calculate_shapley(analysis_data, synergy_results['total_synergy'])
        print("✅ シャプレー値の計算が完了しました。")

        # 6. ビジネス指標予測（このパイプライン内にロジックを統合）
        c_biz = self.config['business_forecast']
        perf_data = []
        for t in range(len(time_axis)):
            perf = solution[t]
            synergy = np.sum(synergy_results['total_synergy'] * np.outer(perf, perf)) / 2
            perf_data.append({
                'time_months': time_axis[t],
                'revenue_growth_rate': np.sum(perf) * c_biz['revenue_base_multiplier'] + synergy * c_biz['revenue_synergy_multiplier'],
                'innovation_index': synergy * c_biz['innovation_synergy_multiplier'],
            })
        business_forecast_df = pd.DataFrame(perf_data)
        print("✅ ビジネス指標の予測が完了しました。")

        # 7. 全ての分析結果を一つのオブジェクトにまとめる
        results = {
            'config': self.config,
            'profiles': profiles,
            'analysis': analysis_data,
            'synergy': synergy_results,
            'simulation_time_axis': time_axis,
            'simulation_solution': solution,
            'shapley_values': shapley_values,
            'business_forecast': business_forecast_df
        }

        # 8. 可視化（Figureオブジェクトの生成のみ）
        results['figures'] = self.viz_generator.generate_all_figures(results)
        print("✅ 全てのグラフオブジェクトの生成が完了しました。")

        # 9. 成果物のファイル保存
        self._save_artifacts(results)

        print("\n--- ✅ パイプライン実行完了 ---")
        return results

    def _save_artifacts(self, results):
        # HTMLファイルの保存
        charts_path = self.config['output']['charts_path']
        if not os.path.exists(charts_path):
            os.makedirs(charts_path)
        for name, fig in results['figures'].items():
            fig.write_html(os.path.join(charts_path, f"{name}.html"), include_plotlyjs='cdn')
        print(f"✅ 全てのグラフをHTMLとして '{charts_path}' に保存しました。")

        # レポートの保存
        names = self.config['data']['display_names']
        report_str = "# 人的シナジー分析レポート V3\n\n"
        report_str += "## 総合シナジー\n"
        report_str += pd.DataFrame(results['synergy']['total_synergy'], columns=names, index=names).to_markdown() + "\n\n"
        report_str += "## 価値貢献度（シャプレー値）\n"
        for name, val in zip(names, results['shapley_values']):
            report_str += f"- {name}: {val:.3f}\n"

        report_path = self.config['output']['report_path']
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"synergy_analysis_report_V3_{timestamp}.md"
        with open(os.path.join(report_path, filename), 'w', encoding='utf-8') as f:
            f.write(report_str)
        print(f"✅ 分析レポートを '{report_path}{filename}' に保存しました。")

# パイプラインを実行
pipeline = AnalysisPipeline(CONFIG)
results = pipeline.run()

✅ ライブラリのインストールが完了しました。
✅ ライブラリのインポートが完了しました。
✅ 分析パラメータの定義が完了しました。
--- パイプライン実行開始 ---
Mounted at /content/drive
✅ 3件のプロフィールを読み込みました。
✅ プロフィール分析（意味ベクトル化）が完了しました。
✅ シナジーマトリクスの計算が完了しました。
✅ 動的成長シミュレーションが完了しました。
✅ シャプレー値の計算が完了しました。
✅ ビジネス指標の予測が完了しました。
✅ 全てのグラフオブジェクトの生成が完了しました。
✅ 全てのグラフをHTMLとして '/content/drive/MyDrive/synergy_analysis_charts_V3/' に保存しました。
✅ 分析レポートを '/content/drive/MyDrive/synergy_analysis_report_V3_20250620_205327.md' に保存しました。

--- ✅ パイプライン実行完了 ---


In [17]:
# ===============================================================
# STEP 10: インタラクティブグラフをHTMLファイルとして保存
# ===============================================================
import os

# 保存先のフォルダを指定 (V2用に新しいフォルダ名にします)
save_path = "/content/drive/MyDrive/synergy_analysis_charts_V2/"

# フォルダが存在しない場合は作成
if not os.path.exists(save_path):
    os.makedirs(save_path)
    print(f"✅ 保存用フォルダを作成しました: {save_path}")

# viz_engineに保存された各グラフをHTMLとして保存
if hasattr(viz_engine, 'charts') and viz_engine.charts:
    print(" V2: グラフのHTMLファイルへの保存を開始します...")
    for chart_name, chart_figure in viz_engine.charts.items():
        file_path = f"{save_path}{chart_name}.html"
        try:
            chart_figure.write_html(file_path, include_plotlyjs='cdn')
            print(f"✅ グラフ '{chart_name}' をHTMLファイルとして保存しました: {file_path}")
        except Exception as e:
            print(f"⚠️ '{chart_name}' の保存中にエラーが発生しました: {e}")
    print("✅ 全てのグラフの保存が完了しました。")
else:
    print("⚠️ 保存するグラフオブジェクトが見つかりませんでした。")

⚠️ 保存するグラフオブジェクトが見つかりませんでした。


In [18]:
# Google Driveへの書き込みテスト用コード

import os

test_folder_path = "/content/drive/MyDrive/synergy_analysis_charts_V2/"
test_file_path = os.path.join(test_folder_path, "test_file.txt")

print(f"テストファイルを作成しようとしています: {test_file_path}")

try:
    # フォルダが存在しない場合は作成
    if not os.path.exists(test_folder_path):
        os.makedirs(test_folder_path)
        print(f"フォルダを作成しました: {test_folder_path}")

    # テストファイルを書き込む
    with open(test_file_path, "w") as f:
        f.write("This is a test file.")

    print(f"✅ テストファイルの書き込みに成功しました。")
    print("Googleドライブの画面（またはColabのファイルブラウザ）を再読み込みして、")
    print("`synergy_analysis_charts_V2`フォルダ内に`test_file.txt`が見えるかご確認ください。")

except Exception as e:
    print(f"⚠️ テストファイルの書き込み中にエラーが発生しました: {e}")

テストファイルを作成しようとしています: /content/drive/MyDrive/synergy_analysis_charts_V2/test_file.txt
✅ テストファイルの書き込みに成功しました。
Googleドライブの画面（またはColabのファイルブラウザ）を再読み込みして、
`synergy_analysis_charts_V2`フォルダ内に`test_file.txt`が見えるかご確認ください。


In [19]:
print("グラフ1/5: 総合シナジーマトリックスの描画テスト...")
viz_engine.create_synergy_heatmap().show()
print("完了")

グラフ1/5: 総合シナジーマトリックスの描画テスト...


完了


In [20]:
print("グラフ2/5: 動的成長軌道の描画テスト...")
viz_engine.create_growth_trajectory().show()
print("完了")

グラフ2/5: 動的成長軌道の描画テスト...


完了


In [21]:
print("グラフ3/5: ビジネス指標ダッシュボードの描画テスト...")
viz_engine.create_business_metrics_dashboard().show()
print("完了")

グラフ3/5: ビジネス指標ダッシュボードの描画テスト...


完了


In [22]:
print("グラフ4/5: シャプレー値の描画テスト...")
viz_engine.create_shapley_value_chart().show()
print("完了")

グラフ4/5: シャプレー値の描画テスト...


完了


In [23]:
print("グラフ5/5: チーム相互作用ネットワークの描画テスト...")
viz_engine.create_network_graph().show()
print("完了")

グラフ5/5: チーム相互作用ネットワークの描画テスト...


完了


In [24]:
# ===============================================================
# STEP 10: 【最終版】可視化とHTMLファイルの保存
# ===============================================================
# 以前の5つの描画セルとSTEP10の保存セルを、この一つのセルにまとめます。

import os

print("📊 V2: 可視化とHTML保存を開始します...")

# 1. グラフオブジェクトを生成し、リストに保存
#    この方法なら、前のセルで何が起きたかを気にする必要がありません。
charts_to_save = {
    'synergy_heatmap': viz_engine.create_synergy_heatmap(),
    'growth_trajectory': viz_engine.create_growth_trajectory(),
    'business_metrics_dashboard': viz_engine.create_business_metrics_dashboard(),
    'shapley_value_chart': viz_engine.create_shapley_value_chart(),
    'network_graph': viz_engine.create_network_graph()
}
print("✅ 5つのグラフオブジェクトの生成が完了しました。")

# 2. 生成した各グラフをノートブック上に表示
print("\n--- グラフの表示を開始 ---")
for name, fig in charts_to_save.items():
    print(f"表示中: {name}")
    fig.show()
print("✅ 全てのグラフの表示が完了しました。")


# 3. HTMLファイルとしてGoogle Driveに保存
save_path = "/content/drive/MyDrive/synergy_analysis_charts_V2/"
if not os.path.exists(save_path):
    os.makedirs(save_path)
    print(f"\n✅ 保存用フォルダを作成しました: {save_path}")

if charts_to_save:
    print("\n--- HTMLファイルへの保存を開始 ---")
    for chart_name, chart_figure in charts_to_save.items():
        file_path = f"{save_path}{chart_name}.html"
        try:
            chart_figure.write_html(file_path, include_plotlyjs='cdn')
            print(f"✅ '{chart_name}' を保存しました: {file_path}")
        except Exception as e:
            print(f"⚠️ '{chart_name}' の保存中にエラーが発生しました: {e}")
    print("✅ 全てのグラフのHTML保存が完了しました。")
else:
    print("⚠️ 保存するグラフがありませんでした。")

📊 V2: 可視化とHTML保存を開始します...
✅ 5つのグラフオブジェクトの生成が完了しました。

--- グラフの表示を開始 ---
表示中: synergy_heatmap


表示中: growth_trajectory


表示中: business_metrics_dashboard


表示中: shapley_value_chart


表示中: network_graph


✅ 全てのグラフの表示が完了しました。

--- HTMLファイルへの保存を開始 ---
✅ 'synergy_heatmap' を保存しました: /content/drive/MyDrive/synergy_analysis_charts_V2/synergy_heatmap.html
✅ 'growth_trajectory' を保存しました: /content/drive/MyDrive/synergy_analysis_charts_V2/growth_trajectory.html
✅ 'business_metrics_dashboard' を保存しました: /content/drive/MyDrive/synergy_analysis_charts_V2/business_metrics_dashboard.html
✅ 'shapley_value_chart' を保存しました: /content/drive/MyDrive/synergy_analysis_charts_V2/shapley_value_chart.html
✅ 'network_graph' を保存しました: /content/drive/MyDrive/synergy_analysis_charts_V2/network_graph.html
✅ 全てのグラフのHTML保存が完了しました。
